In [ ]:
from google.adk.agents import LlmAgent, Agent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner, InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory, google_search, AgentTool
from google.genai import types
from dotenv import load_dotenv
import os

print("✅ ADK components imported successfully.")

In [ ]:
load_dotenv()

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Agent to improvise the user prompt

In [ ]:
prompt_improve_agent = Agent(
    name="prompt_improve_agent",
    model=Gemini(model="gemini-2.5-pro", retry_options=retry_config),
    description="You are an expert prompt optimizer that transforms a simple job search prompt into a detailed, effective prompt.",
    instruction="Your task is to just improvise the given prompt with more detailed and effective words. You are not allowed to search for job openings. While generating the prompt do not any informative text. Stick only to the prompt as I need to pass it to the job_searching_agent. Add a LIMIT of MAX 5 job urls.",
    output_key="improved_prompt",
    # tools=[google_search],
)

## Agent to find the Job URLs


In [50]:
get_urls_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="get_urls_agent",
    description="You acts as a URL Retrieval Specialist dedicated to finding current job openings. The core task is to process a user-provided company name and quickly search for, verify, and return a list of direct URLs to active job postings on that company's official career sites.",
    instruction="""
    You are a job searching agent who is specialized in finding the URLs for the openings. You take the prompt and follow the below listed instructions while find the jobs.

    Prompt: {improved_prompt}

    1. Identify Company & Keywords: Immediately parse the user input to determine the Company Name and any optional Search Keywords (role, skill, location).

    2. Target Careers Page: Execute a search query targeting the company's official Careers Page or Jobs Portal as the primary source.

    3. Apply Filters: Use any provided keywords to filter the search results, aiming for live job listings.

    4. Verify Active Status: Ensure the retrieved URLs lead directly to currently active job postings or search results; discard expired or not found jobs or non-job links (e.g., blogs, news).

    5. Prioritize Official Sources: Favor URLs from the company's official domain or known ATS platforms (Greenhouse, Lever, LinkedIn, etc.).

    6. Strictly Exclude Commentary: Do not generate any introductory text, advice, conversational filler (greetings/closings), or tool names in the final output.

    7. Use Clean List Format: Present the final URLs as a simple, clean, unnumbered list.

    8. New Line per URL: Place each URL on its own line.

    9. Limit Output: Provide a maximum of 5 relevant URLs unless the user explicitly requested more.
    """,
    tools=[google_search],
    output_key="job_opening_urls",
)